# Swiss Prot Sequence To Word2Vec

This demo generates Word2Vector models from protein sequences in UniProt using overlapping n-grams.

![SwissProt](http://swift.cmbi.ru.nl/teach/SWISS/IMAGE/swissprot2.gif)

[UniProt](http://www.uniprot.org/)

## Imports

In [2]:
from pyspark import SparkConf, SparkContext
from mmtfPyspark.ml import proteinSequenceEncoder
from mmtfPyspark.datasets import uniProt

## Configure Spark Context

In [3]:
conf = SparkConf() \
            .setMaster("local[*]") \
            .setAppName("SwissProtSequenceWord2VecEncodeDemo")

sc = SparkContext(conf = conf)

## Get Swiss Prot dataset from UniProt

In [6]:
data = uniProt.get_dataset(uniProt.SWISS_PROT)

# Make sure there are no empty sequence records
data = data.na.drop(subset = ["sequence"])

data.show(10, False)

+---+----------------+----------+------------------------------------------+----------------------------+---------+----------------+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|db |uniqueIdentifier|entryName |proteinName                               |organismName                |geneName |proteinExistence|sequenceVersion|sequence                                                                                                                                                                                                                                            

## Generate Word2Vec model

In [7]:
segmentLength = 11
n = 2
windowSize = (segmentLength - 1)/2
vectorSize = 50

# add Word2Vec encoded feature vector
encoder = proteinSequenceEncoder(data)
data = encoder.overlappingNgramWord2VecEncode(n, windowSize, vectorSize).cache()

## Example of output rows

In [9]:
data.head(2)

[Row(db='>sp', uniqueIdentifier='Q6GZX4', entryName='001R_FRG3G', proteinName='Putativetranscriptionfactor001R', organismName='Frogvirus3(isolateGoorha)', geneName='FV3-001R', proteinExistence=4, sequenceVersion=1, sequence='MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGHFSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTKQHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDFKALVESAHRMRQGHMINVKYILYQLLKKHGHGPDGPDILTVKTGSKGVLYDDSFRKIYTDLGWKFTPL', ngram=['MA', 'AF', 'FS', 'SA', 'AE', 'ED', 'DV', 'VL', 'LK', 'KE', 'EY', 'YD', 'DR', 'RR', 'RR', 'RR', 'RM', 'ME', 'EA', 'AL', 'LL', 'LL', 'LS', 'SL', 'LY', 'YY', 'YP', 'PN', 'ND', 'DR', 'RK', 'KL', 'LL', 'LD', 'DY', 'YK', 'KE', 'EW', 'WS', 'SP', 'PP', 'PR', 'RV', 'VQ', 'QV', 'VE', 'EC', 'CP', 'PK', 'KA', 'AP', 'PV', 'VE', 'EW', 'WN', 'NN', 'NP', 'PP', 'PS', 'SE', 'EK', 'KG', 'GL', 'LI', 'IV', 'VG', 'GH', 'HF', 'FS', 'SG', 'GI', 'IK', 'KY', 'YK', 'KG', 'GE', 'EK', 'KA', 'AQ', 'QA', 'AS', 'SE', 'EV', 'VD', 'DV',

## Terminate Spark Context

In [10]:
sc.stop()